In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
independent.shape

(400, 3)

In [12]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dependent=dataset[[ 'Purchased',]]

In [14]:
from sklearn.model_selection import train_test_split    
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
from sklearn.model_selection import GridSearchCV                
param_grid = {'alpha': [1,2,3,4,5,5],
              'force_alpha':[True,False],
              'fit_prior':[True,False],        
              'class_prior':[None]}

grid = GridSearchCV(MultinomialNB(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted') 
   
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\sabeen\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5, 5], 'class_prior': [None],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [17]:
re=grid.cv_results_

grid_prediction=grid.predict(x_test)



In [18]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[68 11]
 [28 13]]


In [19]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       0.71      0.86      0.78        79
           1       0.54      0.32      0.40        41

    accuracy                           0.68       120
   macro avg       0.62      0.59      0.59       120
weighted avg       0.65      0.68      0.65       120



In [20]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}: 0.6482857142857144


In [21]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[68 11]
 [28 13]]


In [22]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.71      0.86      0.78        79
           1       0.54      0.32      0.40        41

    accuracy                           0.68       120
   macro avg       0.62      0.59      0.59       120
weighted avg       0.65      0.68      0.65       120



In [23]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.5152824945970979

In [24]:
table=pd.DataFrame.from_dict(re)
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004859,0.001714,0.015928,0.002333,1,None,True,True,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
1,0.007817,0.002350,0.011748,0.006444,1,None,True,False,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
2,0.004198,0.002463,0.008567,0.002564,1,None,False,True,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,15
3,0.006334,0.003330,0.009090,0.001007,1,None,False,False,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,15
4,0.007552,0.003909,0.006760,0.001752,2,None,True,True,"{'alpha': 2, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
5,0.005036,0.001580,0.008415,0.003047,2,None,True,False,"{'alpha': 2, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
6,0.005511,0.005128,0.009162,0.001929,2,None,False,True,"{'alpha': 2, 'class_prior': None, 'fit_prior':...",0.544700,0.472527,0.540136,0.578065,0.558458,0.538777,0.035656,13
7,0.005539,0.002554,0.006579,0.002132,2,None,False,False,"{'alpha': 2, 'class_prior': None, 'fit_prior':...",0.544700,0.472527,0.540136,0.578065,0.558458,0.538777,0.035656,13
8,0.008809,0.002486,0.006322,0.002799,3,None,True,True,"{'alpha': 3, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
9,0.002617,0.002581,0.008034,0.002802,3,None,True,False,"{'alpha': 3, 'class_prior': None, 'fit_prior':...",0.618850,0.503106,0.616071,0.609571,0.619155,0.593351,0.045254,1
